<a href="https://colab.research.google.com/github/Montse1708/Dashboard/blob/main/TI_1_Practica4_JessicaMontserratMoralesEnrique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 4

**Nombre:** Jessica Montserrat Morales Enrique  
**e-mail:** jessica.morales5556@alumnos.udg.mx

## MODULES

In [ ]:
import panel as pn
import panel.widgets as pnw

pn.extension('plotly')

import pandas as pd
import numpy as np

import plotly.graph_objects as go

import math
from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

##CLASSES

In [ ]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

##FUNCTIONS

###Brownian Motion (BM)

In [ ]:
###############################################################################################
# Brownian Motion Trajectory
###############################################################################################
def BM_2d(n_steps=1000, speed=6, s_x_pos=0, s_y_pos=0):
    """
    Arguments:
        n_steps:
        speed:
        s_pos:
    Returns:
        BM_2d_df:
    """
    # Init velocity vector
    velocity = Vec2d(speed,0)

    BM_2d_df = pd.DataFrame(columns = ['x_pos','y_pos'])
    temp_df = pd.DataFrame([{'x_pos': s_x_pos, 'y_pos': s_y_pos}])
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

    for i in range(n_steps-1):
        turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
        velocity = velocity.rotated(turn_angle)

        temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x, 'y_pos': BM_2d_df.y_pos[i]+velocity.y}])
        BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

    return BM_2d_df

In [ ]:
#Verificar que la función es correcta con el gráfico de la Brownian Motion

#Plot
BM_2d_df_2 = BM_2d(2000, 9, 5, 7)

#Init figure
fig_BM_2d = go.Figure()

#Plot trajectory
fig_BM_2d.add_trace(go.Scatter(
                      x = BM_2d_df_2.x_pos,
                      y = BM_2d_df_2.y_pos,
                      marker = dict(size=2),
                      line = dict(width=1),
                      mode='lines',
                      name='BM_2d_trajectory',
                      showlegend=True
))

#Actualizar el diseño para agregar el título
fig_BM_2d.update_layout(title='Trayectoria de Brownian Motion en 2D')

#Mostrar el gráfico
fig_BM_2d.show()

###Correlated Random Walk (CRW)

In [152]:
######################################################################
# Correlated Random Walks (CRW)
######################################################################
def CRW_2d(CRW_exponents=0.2, resolution=800, speed=6, x_pos=0, y_pos=0):
  aux_domain = np.linspace(0, 2*np.pi, resolution)
  #init velocity vector
  velocity = Vec2d(speed, 0)
  wrapcauchy_rvs = wrapcauchy.rvs(CRW_exponents, size=resolution)
  #init DataFrames
  CRW_2d_df_s = pd.DataFrame(columns=['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': x_pos, 'y_pos': y_pos}])

  CRW_2d_df_s = pd.concat([CRW_2d_df_s,temp_df], ignore_index=True)

  for j in range(resolution):
    velocity = velocity.rotated(wrapcauchy_rvs[j])

    temp_df = pd.DataFrame([{'x_pos':CRW_2d_df_s.x_pos[j]+velocity.x,'y_pos':CRW_2d_df_s.y_pos[j]+velocity.y}])
    CRW_2d_df_s = pd.concat([CRW_2d_df_s,temp_df], ignore_index=True)

  return CRW_2d_df_s

In [123]:
# Verificar que la función es correcta con el gráfico de Correlated Random Walk (CRW)
CRW_df = CRW_2d(0.8, 500, 5, 5, 7)

# Inicializar el gráfico
fig_CRW = go.Figure()

# Agregar la trayectoria al gráfico
fig_CRW.add_trace(go.Scatter(
    x = CRW_df.x_pos,
    y = CRW_df.y_pos,
    marker = dict(size=2),
    line = dict(width=1),
    mode='lines',
    name='CRW_trajectory',
    showlegend=True
))

# Actualizar el diseño para agregar el título
fig_CRW.update_layout(title='Trayectoria de Correlated Random Walk (CRW) en 2D')

# Mostrar el gráfico
fig_CRW.show()

###Lévy Flight (LF)

In [133]:
######################################################################
# Lévy Flight (LF)
######################################################################
def LF_2d(Levy_exponent=0.7, miu=3.0, beta=0, n_steps=250, x_pos=0, y_pos=0):
  #Init variables
  Levy_rvs = levy_stable.rvs(Levy_exponent,beta, loc=miu, size=n_steps)

  Wrapcauchy_rvs = wrapcauchy.rvs(c=Levy_exponent, loc=beta, size=n_steps)
  cauchy_rvs = Wrapcauchy_rvs
  for n in range(len(cauchy_rvs)):
    for m in range(n):
      cauchy_rvs[n] += cauchy_rvs[m]

  Levy_rvs_abs = abs(Levy_rvs)

  LF_2d_df = pd.DataFrame(columns=['x_pos', 'y_pos'])

  temp_df = pd.DataFrame([{'x_pos':x_pos,'y_pos':y_pos}])

  LF_2d_df = pd.concat([LF_2d_df,temp_df], ignore_index=True)

  for i in range(n_steps-1):
    #init velocity vector
    velocity = Vec2d(Levy_rvs_abs[i], 0)
    velocity = velocity.rotated(cauchy_rvs[i])

    temp_df = pd.DataFrame([{'x_pos':LF_2d_df.x_pos[i]+velocity.x,'y_pos':LF_2d_df.y_pos[i]+velocity.y}])
    LF_2d_df = pd.concat([LF_2d_df,temp_df], ignore_index=True)
  return LF_2d_df

In [ ]:
# Verificamos que la función LF_2d funcione correctamente
LF_2d_df = LF_2d(0.8, 1000, 5, 5, 7, 0.7, 1.0, 3.0)

# Creamos una figura de Plotly
fig_LF_2d = go.Figure()

# Agregamos la trayectoria de la caminata de Lévy a la figura
fig_LF_2d.add_trace(go.Scatter(
    x=LF_2d_df.x_pos,
    y=LF_2d_df.y_pos,
    marker=dict(size=2),
    line=dict(width=1),
    mode='lines',
    name='LF_2d',
    showlegend=True
))

# Actualizamos el diseño de la figura con un título
fig_LF_2d.update_layout(title_text='Levy flight in 3D')

# Mostramos la figura
fig_LF_2d.show()

##METRICAS

In [ ]:
def distance_euclidean(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

###Path length

In [ ]:
def path_length(trajectory):
  n_steps = trajectory.shape[0]
  array_trajectory = [];
  for i in range(1, n_steps):
    array_trajectory.append(distance_euclidean(trajectory.iloc[i-1], trajectory.iloc[i]))
    path_length_df = pd.DataFrame(np.cumsum(array_trajectory), columns=['PL'])
  return path_length_df

In [134]:
def path_length(df = pd.DataFrame):
  n_df = np.array([distance_euclidean(df.loc[i-1], df.iloc[i]) for i in range(1,df.shape[0])])
  return np.cumsum(n_df)

###Mean Squared Displacement

In [139]:
def MSD(df = pd.DataFrame):
  msd_2 = np.empty(shape=(0))
  list_msd = []

  for i in range(1,df.shape[0]):
    disp_vec = np.array([distance_euclidean
     (df.iloc[n-i], df.iloc[n]) for n in range(i, df.shape[0],1)])
    list_msd.append(np.sum(np.power(disp_vec, 2))/len(disp_vec))

  msd_2 = np.asarray(list_msd)

  return msd_2

##DASHBOARD

In [154]:
opcion = pnw.RadioButtonGroup(name='opcion', options=['BM','CRW','LF'])
size = 150
n_steps = pnw.IntSlider(name='Number of steps', width=size, value=0, step=20, start=0, end=1000)
cauchy = pnw.FloatSlider(name='Coeficent', width=size, value=0.02, step=0.01, start=0.02, end=0.99)
alpha = pnw.IntSlider(name='beta', width=size, value=0, step=1, start=-3, end=3)
x_pos = pnw.IntInput(name='X position', width=size,value=0, step=1, start=0, end=25)
y_pos = pnw.IntInput(name='Y position', width=size,value=0, step=1, start=0, end=25)
speed = pnw.IntSlider(name='Speed', width=size, value=0, step=1, start=0, end=20)
metric = pnw.Select(name='Metric', width=size,value='Path Length', options=['Path Length','MSD'])


@pn.depends(opcion)
def opc(opcion):
  if opcion == 'BM':
    return pn.Column(n_steps, x_pos, y_pos, speed, metric)
  elif opcion == 'CRW':
    return pn.Column(n_steps, x_pos, y_pos, speed, cauchy, metric)
  elif opcion == 'LF':
    return pn.Column(n_steps, x_pos, y_pos, cauchy, alpha, metric)

@pn.depends(opcion)
def opc2(opcion):
  if opcion == 'BM':
    return BW
  elif opcion == 'CRW':
    return crw
  elif opcion == 'LF':
    return lf

@pn.depends(n_steps, x_pos, y_pos, speed)
def BW(n_steps, x_pos, y_pos, speed):
  bm_df_1 = BM_2d(n_steps,speed,x_pos,y_pos)
  fig_BM = go.Figure()
  fig_BM.add_trace(go.Scatter3d(
      x=bm_df_1.x_pos,
      y=bm_df_1.y_pos,
      z=np.linspace(0,1,n_steps),
      marker=dict(size=2),
      line=dict(width=2),
      mode='lines',
      name='BM',
      showlegend=True
  ))

  fig_BM.update_layout(title_text='Brownian Walker in 3D',
                        autosize=False,
                        width = 600,
                        height = 500)
  return fig_BM

@pn.depends(n_steps, cauchy, x_pos, y_pos, speed)
def crw(n_steps, cauchy, x_pos, y_pos, speed):
  random = CRW_2d(cauchy,n_steps, speed, x_pos, y_pos)
  fig_CRW = go.Figure()

  fig_CRW.add_trace(go.Scatter3d(
      x=random.x_pos,
      y=random.y_pos,
      z=np.linspace(0,1,n_steps),
      marker=dict(size=2),
      line=dict(width=2),
      mode='lines',
      name=f'CRW {cauchy}',
      showlegend=True
  ))
  fig_CRW.update_layout(title_text='Correlated Random Walk in 3D',
                        autosize=False,
                        width = 600,
                        height = 500)
  return fig_CRW

pn.Row(pn.Column(opcion, opc),opc2)

Row
    [0] Column
        [0] RadioButtonGroup(name='opcion', options=['BM', 'CRW', 'LF'], value='BM')
        [1] ParamFunction(function, _pane=Column, defer_load=False)
    [1] ParamFunction(function, _pane=ParamFunction, defer_load=False)